In [1]:
from google.genai import types
from google.adk.agents import Agent
from google.adk.runners import InMemoryRunner
from google.adk.models.google_llm import Gemini
from google.adk.agents.remote_a2a_agent import RemoteA2aAgent, AGENT_CARD_WELL_KNOWN_PATH

In [2]:
retry_config=types.HttpRetryOptions(attempts=3, exp_base=5, initial_delay=1, http_status_codes=[429, 500, 503, 504])
model = Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config)

In [3]:
%%writefile /tmp/a2a_server.py

from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.a2a.utils.agent_to_a2a import to_a2a

validation_agent = Agent(
    name="ValidationAgent",
    model=Gemini(model="gemini-2.5-flash-lite"),
    instruction="""
    You are a video idea validator for short videos on social media targeted to kids.
    When asked about an idea:
    - If the idea is appropriate for small children, you MUST respond with 'APPROPRIATE'.
    - Otherwise, you MUST respond with 'INAPPROPRIATE'. 
    """,
)

app = to_a2a(validation_agent, port=8001)

Writing /tmp/a2a_server.py


Run in separate terminal, Jupyter cannot handle remote agents with another loop. 

In [ ]:
# !GOOGLE_API_KEY=... uvicorn a2a_server:app --host 127.0.0.1 --port 8001 --app-dir /tmp

/tmp/a2a_server.py:17: UserWarning: [EXPERIMENTAL] to_a2a: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  app = to_a2a(validation_agent, port=8001)
/home/kamil/main/TODO/polibuda/Studia/articles/Articles/AI/Google Agent Development Kit/.venv/lib/python3.10/site-packages/google/adk/a2a/utils/agent_to_a2a.py:126: UserWarning: [EXPERIMENTAL] A2aAgentExecutor: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  agent_executor = A2aAgentExecutor(
/home/kamil

In [8]:
remote_validation_agent = RemoteA2aAgent(
    name="RemoteValidationAgent",
    agent_card=f"http://localhost:8001/{AGENT_CARD_WELL_KNOWN_PATH}",
)

/tmp/ipykernel_41532/2309621071.py:1: UserWarning: [EXPERIMENTAL] RemoteA2aAgent: ADK Implementation for A2A support (A2aAgentExecutor, RemoteA2aAgent and corresponding supporting components etc.) is in experimental mode and is subjected to breaking changes. A2A protocol and SDK arethemselves not experimental. Once it's stable enough the experimental mode will be removed. Your feedback is welcome.
  remote_validation_agent = RemoteA2aAgent(


In [9]:
idea_agent = Agent(
    name="IdeaAgent",
    model=model,
    instruction="""
    You are a creative idea generator for short videos for social media.
    Create unique and innovative idea based on user prompts.
    Before returning the idea, you MUST first consult the RemoteValidationAgent to ensure the idea is appropriate for small children.
    Return the idea and the validation result.
    """,
    sub_agents=[remote_validation_agent],
)

In [10]:
runner = InMemoryRunner(agent=idea_agent)
_ = await runner.run_debug("Create a short video about puppies.")


 ### Created new session: debug_session_id

User > Create a short video about puppies.
RemoteValidationAgent > APPROPRIATE
